In [2]:
import sys
sys.path.append(".")
sys.path.append("../")

* Building parser, but really cheating, because I'm loading cached data in the `Built Labeling Rules` step

In [3]:
from CCG_new import parser

In [4]:
trainer = parser.CCGParserTrainer("re", explanation_file="../data/tacred_explanations.json",
                                  unlabeled_data_file="../data/tacred_dev.json")

In [5]:
trainer.train()

Parser: Loaded explanation data
Parser: Created and Set Grammar
Parser: Tokenized Explanations
Parser: Built Labeling Rules
Parser: Set labeling functions
Parser: Built soft labeling functions
Parser: Done


In [6]:
parser = trainer.get_parser()

In [7]:
loaded_data = parser.loaded_data

* Number of explanations with soft_matching functions

In [8]:
len(parser.filtered_raw_explanations)

157

* The next three cells are showing you the three main datastructures that you can use to understand what the soft-matching functions are

In [9]:
parser.soft_labeling_functions

[(<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'per:children'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'per:date_of_birth'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'org:parents'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'per:cause_of_death'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'per:stateorprovinces_of_residence'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'per:other_family'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'org:stateorprovince_of_headquarters'),
 (<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>,
  'per:alternate_names'),
 (<function CCG_new.constants.<

In [10]:
parser.filtered_raw_explanations

{('.root',
  ('@And',
   ('@Is',
    'There',
    ('@AtMost',
     ('@between', ('@And', 'ArgY', 'ArgX')),
     ('@Num', '3', 'tokens'))),
   ('@Is',
    ('@Word', "'s daughter"),
    ('@between',
     ('@And',
      'ArgY',
      'ArgX'))))): 'The phrase "\'s daughter" links SUBJ and OBJ and there are no more than three words between SUBJ and OBJ',
 ('.root',
  ('@And',
   ('@Is',
    'There',
    ('@AtMost',
     ('@between', ('@And', 'ArgY', 'ArgX')),
     ('@Num', '3', 'tokens'))),
   ('@Is',
    ('@Word', 'was born'),
    ('@between',
     ('@And',
      'ArgY',
      'ArgX'))))): 'SUBJ and OBJ sandwich the phrase "was born" and there are no more than three words between SUBJ and OBJ',
 ('.root',
  ('@And',
   ('@Is', 'as part of its', ('@between', ('@And', 'ArgY', 'ArgX'))),
   ('@Is',
    'There',
    ('@AtMost',
     ('@between', ('@And', 'ArgY', 'ArgX')),
     ('@Num',
      '5',
      'tokens'))))): 'There are no more than five words between SUBJ and OBJ and "as part of its" 

In [11]:
parser.soft_label_function_to_semantic_map

{<function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>: ('.root',
  ('@And',
   ('@Is',
    'There',
    ('@AtMost',
     ('@between', ('@And', 'ArgY', 'ArgX')),
     ('@Num', '3', 'tokens'))),
   ('@Is', ('@Word', "'s daughter"), ('@between', ('@And', 'ArgY', 'ArgX'))))),
 <function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>: ('.root',
  ('@And',
   ('@Is',
    'There',
    ('@AtMost',
     ('@between', ('@And', 'ArgY', 'ArgX')),
     ('@Num', '3', 'tokens'))),
   ('@Is', ('@Word', 'was born'), ('@between', ('@And', 'ArgY', 'ArgX'))))),
 <function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dict, mask_mat)>: ('.root',
  ('@And',
   ('@Is', 'as part of its', ('@between', ('@And', 'ArgY', 'ArgX'))),
   ('@Is',
    'There',
    ('@AtMost',
     ('@between', ('@And', 'ArgY', 'ArgX')),
     ('@Num', '5', 'tokens'))))),
 <function CCG_new.constants.<lambda>.<locals>.<lambda>(label_mat, keyword_dic

In [7]:
import json
import pickle
import dill

* Loading the original explanation file

In [12]:
with open("../data/tacred_explanations.json") as f:
    explanations = json.load(f)
    explanations = [ent["explanation"] for ent in explanations]

In [29]:
explanations[120]

"The word 'employees' appears right after OBJ"

* Figuring out the index of each soft_labebling_function

In [14]:
labeling_function_indicies = []
for pair in parser.soft_labeling_functions:
    func, _ = pair
    sem = parser.soft_label_function_to_semantic_map[func]
    expl = parser.filtered_raw_explanations[sem]
    index = explanations.index(expl)
    labeling_function_indicies.append(index)
    

In [15]:
len(labeling_function_indicies)

157

In [16]:
labeling_functions = parser.labeling_functions
function_ner_types = parser.ner_types
task = "re"

* Commented out the below, but basically this shows you the logic I use to do strict matching

In [17]:
# with open("../data/training_data/train_phrases_debug.p", "rb") as f:
#         phrases = pickle.load(f)

        
# matched_data_indicies = []
# soft_matched_indicies = []
# # matched_data_tuples = []
# # unlabeled_data_phrases = []

# for i, phrase in enumerate(phrases):
#     not_matched = True
#     for function in labeling_functions:
#         try:
#             if function(phrase):
#                 if task == "re":
#                     subj_type = phrase.ners[phrase.subj_posi].lower()
#                     obj_type = phrase.ners[phrase.obj_posi].lower()
#                     if function_ner_types[function][0] == subj_type and function_ner_types[function][1] == obj_type:
#                         sentence = phrase.sentence.replace("SUBJ", "SUBJ-{}".format(phrase.ners[phrase.subj_posi]))
#                         sentence = sentence.replace("OBJ", "OBJ-{}".format(phrase.ners[phrase.obj_posi]))
# #                         matched_data_tuples.append((sentence, labeling_functions[function]))
#                         matched_data_indicies.append(i)
#                         not_matched = False
#                         break
#                 else:
# #                     matched_data_tuples.append((phrase.sentence, labeling_functions[function]))
#                     matched_data_indicies.append(i)
#                     not_matched = False
#                     break
#         except:
#             continue
#     if not_matched:
#         soft_matched_indicies.append(i)

In [18]:
# with open("matching_indxes.json", "w") as f:
#     json.dump({
#         "matched_data_indicies" : matched_data_indicies, 
#         "soft_matched_indicies" : soft_matched_indicies
#     }, f)

* Showing you the result of strict_matching, we differ a little, but not much (maybe like 20 sentences)

In [8]:
with open("matching_indxes.json") as f:
    data = json.load(f)
    matched_data_indicies = data["matched_data_indicies"]
    soft_matched_indicies = data["soft_matched_indicies"]

In [9]:
len(matched_data_indicies), len(soft_matched_indicies)

(2745, 65379)

In [20]:
import torch
from models import Find_Module
from training.util_functions import build_custom_vocab
from training.util_functions import set_re_dataset_ner_label_space
from tqdm import tqdm
from training.util_classes import BaseVariableLengthDataset
from training.train_util_functions import build_phrase_input, build_mask_mat_for_batch, batch_type_restrict_re
from training.constants import TACRED_ENTITY_TYPES, TACRED_NERS
from CCG_new.soft_grammar_functions import NER_LABEL_SPACE

* Loading data that gets sent into the FIND MODULE and the data that needs to be sent into soft_matching_functions

In [21]:
save_string = "tacred_glove.840B.300d_-1_-1.0"

with open("../data/training_data/unlabeled_data_{}.p".format(save_string), "rb") as f:
    unlabeled_data = pickle.load(f)

with open("../data/vocabs/vocab_tacred_glove.840B.300d_-1_0.6.p", "rb") as f:
    vocab = pickle.load(f)

with open("../data/training_data/labeling_functions_{}.p".format(save_string), "rb") as f:
    soft_labeling_functions_dict = dill.load(f)

with open("../data/training_data/query_tokens_{}.p".format(save_string), "rb") as f:
    tokenized_queries = pickle.load(f)
    
with open("../data/training_data/word2idx_{}.p".format(save_string), "rb") as f:
    quoted_words_to_index = pickle.load(f)

pad_idx = vocab["<pad>"]

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

custom_vocab = build_custom_vocab("tacred", len(vocab))
custom_vocab_length = len(custom_vocab)

find_module = Find_Module.Find_Module(vocab.vectors, pad_idx, 300, 300,
                                      torch.cuda.is_available(),
                                      custom_token_count=custom_vocab_length)

find_module.load_state_dict(torch.load("../data/saved_models/Find-Module-pt_official.p"))

<All keys matched successfully>

* Very Important Step, this sets the NER_LABEL_SPACE constant that is used in soft_matching
https://github.com/Rahul-Khanna/NExT/blob/rahul/rahul_code/CCG_new/soft_grammar_functions.py#L14

In [22]:
set_re_dataset_ner_label_space("tacred", TACRED_NERS)

In [23]:
NER_LABEL_SPACE

{'': 0,
 'NUMBER': 1,
 'DATE': 2,
 'EVENT': 3,
 'FAC': 4,
 'STATE_OR_PROVINCE': 5,
 'LANGUAGE': 6,
 'LAW': 7,
 'LOCATION': 8,
 'MONEY': 9,
 'NORP': 10,
 'ORDINAL': 11,
 'ORGANIZATION': 12,
 'PERCENT': 13,
 'PERSON': 14,
 'PRODUCT': 15,
 'QUANTITY': 16,
 'TIME': 17,
 'WORK_OF_ART': 18,
 '<PAD>': 19,
 'TITLE': 20,
 'COUNTRY': 21,
 'CITY': 22,
 'MISC': 23,
 'DURATION': 24,
 'NATIONALITY': 25,
 'CAUSE_OF_DEATH': 26,
 'CRIMINAL_CHARGE': 27,
 'RELIGION': 28,
 'URL': 29,
 'IDEOLOGY': 30}

* More data prep and some constant declarations

In [24]:
find_module = find_module.to(device)

# Get queries ready for Find Module
lfind_query_tokens, _ = BaseVariableLengthDataset.variable_length_batch_as_tensors(tokenized_queries, pad_idx)
lfind_query_tokens = lfind_query_tokens.to(device).detach()

soft_labeling_functions = soft_labeling_functions_dict["function_pairs"]
soft_labeling_function_labels = soft_labeling_functions_dict["labels"]

relation_ner_types = TACRED_ENTITY_TYPES
lower_bound = -20.0

* Main function, for each batch (right now just size 1 for debugging), compute soft-matching socres per function store the results in function_scores, where key - function_index, value - tensor of shape (1, DatasetLength)

In [33]:
function_scores = {}
for i, batch in enumerate(tqdm(unlabeled_data.as_batches(batch_size=1, shuffle=False))):
        unlabeled_tokens, unlabeled_token_lengths, phrases = batch
        unlabeled_tokens = unlabeled_tokens.to(device)
        phrase_input = build_phrase_input(phrases, pad_idx, "re").to(device).detach()
        b_size, seq_length = unlabeled_tokens.shape
        mask_mat = build_mask_mat_for_batch(seq_length).to(device).detach()
        with torch.no_grad():
            lfind_output = find_module.soft_matching_forward(unlabeled_tokens.detach(), lfind_query_tokens, lower_bound).detach() # B x seq_len x Q

            function_batch_scores = []
            for i, pair in enumerate(soft_labeling_functions):
                func, rel = pair
                func_index = labeling_function_indicies[i]
                if func_index == 120:
                    batch_scores = func(lfind_output, quoted_words_to_index, mask_mat)(phrase_input).detach() # 1 x B

                    type_restrict_multiplier = batch_type_restrict_re(rel, phrase_input, relation_ner_types).detach() # 1 x B
                    final_scores = batch_scores * type_restrict_multiplier # 1 x B
                    if func_index in function_scores:
                        function_scores[func_index] = torch.cat([function_scores[func_index], final_scores], dim=1)
                    else:
                        function_scores[func_index] = final_scores

53it [00:17,  2.96it/s]


IndexError: index 26 is out of bounds for dimension 0 with size 26

* To help debug this index issue, I placed this check in the creation of phrase objects... making sure subj_posi and obj_posi are always less than the length of sentence https://github.com/Rahul-Khanna/NExT/blob/rahul/rahul_code/training/train_util_functions.py#L40

* Finally, `unlabeled_data` is an object of type https://github.com/Rahul-Khanna/NExT/blob/rahul/rahul_code/training/util_classes.py#L149, so you can access the tokens array and/or phrase array like `unlabeled_data.tokens[i]` or `unlabeled_data.phrases[i]` (if you'd like to)